In [6]:
#docker run \
#    --name neohype --publish=7474:7474 --publish=7687:7687 \
#    --volume=$HOME/neo4j/data:/data \
#    neo4j

In [7]:
#docker run -it --rm \
#  --publish=7474:7474 --publish=7687:7687 \
#  --user="$(id -u):$(id -g)" \
#  -e NEO4J_AUTH=none \
#  --env NEO4JLABS_PLUGINS='["graph-data-science"]' \
#  neo4j:4.2

In [8]:
from flask import Flask, request, jsonify, redirect, render_template
from neo4j import GraphDatabase
import csv

In [9]:
with open("cred.txt") as f1:
    data = csv.reader(f1, delimiter=",")
    for row in data:
        username = row[0]
        password = row[1]
        uri = row[2]
        
print(username, password, uri)

neo4j j4neo bolt://0.0.0.0:7687


In [10]:
# Establish connection
driver = GraphDatabase.driver(uri=uri, auth=(username,password))
session = driver.session()

ServiceUnavailable: Connection to 0.0.0.0:7687 closed without handshake response

----

In [ ]:
## Create Flask API
api = Flask(__name__)

In [ ]:
# CREATE STOCK
@api.route("/stock/<string:ticker>", methods=["POST"])
def create_stock(ticker):
    q1 = """
    create (s:stock{ticker:$ticker})
    """
    map = {"ticker": ticker}
    try:
        session.run(q1, map)
        return(f"stock node is created with ticker name = {ticker}")
    except Exception as e:
        return (str(e))

In [ ]:
#create_stock("GME")

In [ ]:
# CREATE SUBREDDIT
@api.route("/subreddit/<string:name>", methods=["POST"])
def create_subreddit(name):
    q1 = """
    create (sr:subreddit{name:$name})
    """
    map = {"name": name}
    try:
        session.run(q1, map)
        return(f"subreddit node is created with subreddit name = {name}")
    except Exception as e:
        return (str(e))

In [ ]:
# CREATE SUBMISSION
@api.route("/submission/<string:subreddit>/<string:date>/<string:id>", methods=["POST"])
def create_submission(subreddit, date, id):
    q1 = '''
    MERGE (sm:submission:'''+subreddit+'''{date:$date, id:$id})
   '''
    map = {"date": date, "id": id}
    try:
        session.run(q1, map)
        return(f"submission node is created with id = {id}")
    except Exception as e:
        return (str(e))

In [ ]:
# CREATE COMMENT
@api.route("/comment/<string:subreddit>/<string:date>/<string:comment_id>/<string:parent_id>", methods=["POST"])
def create_comment(subreddit, date, comment_id, parent_id):
    q1 = '''
    MERGE (c:comment:'''+subreddit+'''{date:$date, comment_id:$comment_id, parent_id:$parent_id})
   '''
    map = {"date": date, "comment_id": comment_id, "parent_id": parent_id}
    try:
        session.run(q1, map)
        return(f"comment node is created with id = {comment_id}")
    except Exception as e:
        return (str(e))

In [ ]:
# CREATE MENTIONS RELATIONSHIP
            
@api.route("/connection/mentions/<string:node_kind>/<string:node_id>/<string:ticker>/<string:date>", methods=["POST"])
def create_mentions_connection(node_kind, node_id, ticker, date):
    q1 = """
    MERGE (s:stock{ticker:'"""+ticker+"""'})
    """
    
    q_submission = """
    MATCH (a:submission),(b:stock)
    WHERE a.id = '"""+node_id+"""' AND b.ticker = '"""+ticker+"""'
    MERGE (a)-[m:MENTIONS {date: '"""+date+"""'}]->(b)
    """
    
    q_comment = """
    MATCH (a:"""+node_kind+"""),(b:stock)
    WHERE a.comment_id = '"""+node_id+"""' AND b.ticker = '"""+ticker+"""'
    MERGE (a)-[m:MENTIONS {date: '"""+date+"""'}]->(b)
    """
    
    map = {"node_kind": node_kind, "node_id": node_id, "ticker": ticker}
    query = q_comment
    if node_kind == 'submission':
        query = q_submission
    try:
        session.run(q1)
        session.run(query, map)
        return(f"mention relationship between {node_id} and {ticker} created with date: {date}")
    except Exception as e:
        return (str(e))


In [ ]:
# CREATE REPLIES RELATIONSHIP
@api.route("/connection/replies/<string:node_kind>/<string:reply_from_id>/<string:reply_to_id>", methods=["POST"])
def create_replies_connection(node_kind, reply_from_id, reply_to_id):
    # reply if to submission
    q_submission = """
    MATCH 
        (cfrom:comment {comment_id: '"""+reply_from_id+"""'}),
        (cto:submission {id: '"""+reply_to_id+"""'})
    MERGE (cfrom)-[r:REPLIES]->(cto)
    """
    # reply if to comment
    q_comment = """
    MATCH 
        (cfrom:comment {comment_id: '"""+reply_from_id+"""'}),
        (cto:comment {comment_id: '"""+reply_to_id+"""'})
    MERGE (cfrom)-[r:REPLIES]->(cto)
    """
    map = {"node_kind":node_kind,"reply_from_id": reply_from_id, "reply_to_id":reply_to_id}
    query = q_comment
    if node_kind == 'submission':
        query = q_submission
    try:
        session.run(query, map)
        return(f"connection 'replied' is created")
    except Exception as e:
        return (str(e))

## Fetch

In [ ]:
@api.route("/subreddits", methods=["GET"])
def get_subreddits():
    q1="""
    match (sr) return sr.name as name
    """
    results = session.run(q1)
    data =  results.data()
    return(jsonify(data))

In [ ]:
## COUNT MENTIONS ON DATE
@api.route("/mentions/<string:ticker>/<string:date>", methods=["GET"])
def get_mentions_on_date(ticker, date):
    query="""
    MATCH ()-[m:MENTIONS]->(s:stock)
    WHERE s.ticker = '"""+ticker+"""' AND m.date = '"""+date+"""'
    RETURN count(m) as count
    """
    map = {"ticker":ticker,"date": date}
    
    try:
        result = session.run(query, map)
        result = result.single()[0]
        result = str(result)
        return result
    except Exception as e:
        return (str(e))

In [ ]:
## COUNT MENTIONS ON DATE
@api.route("/topstocks", methods=["GET"])
def get_top_stocks():
    query="""
    MATCH ()-[r:MENTIONS]->(n)
    RETURN n, count(r) AS num
    ORDER BY num DESC
    """

    try:
        result = session.run(query)
    except Exception as e:
        return (str(e))
    


In [ ]:
if __name__ == "__main__":
    api.run(port=5050)

In [ ]:
i = get_top_stocks()
i